In [1]:
# Setup logging
from timeseria import logger
logger.setup(level='INFO')

⚠️ This Notebook renders with limited features (e.g. no plots) if opened on GitHub, or as an untrusted Notebook in your local environment. Please either open it using [NBViewer](https://nbviewer.org/github/sarusso/Timeseria-notebooks/tree/master/notebooks/) or mark it as [trusted](https://stackoverflow.com/questions/44943646/jupyter-notebook-not-trusted).


# Timeseria quickstart

Timeseria is an object-oriented time series processing library which aims at making it easy to manipulate time series data and to build statistical and machine learning models on top of it.

It comes with a built-in set of common operations (resampling, aggregation, differencing etc.) and models (reconstruction, forecasting and anomaly detection) which can cover a variety of simple use-cases. Moreover, both custom operations and models can be easily plugged in.

Timeseria also addresses by design all those annoying things which are often left as an implementation detail but that actually cause wasting massive amounts of time - as handling data losses, non-uniform sampling rates, differences between time-slotted (aggregated) data and punctual observations, variable time units, timezones, DST changes and so on.

This is a quickstart, if you are looking for a more structured introduction, have a look at the [welcome notebook](https://nbviewer.org/github/sarusso/Timeseria-notebooks/blob/master/notebooks/Welcome.ipynb). Also the [reference documentation](https://timeseria.readthedocs.io) might be useful.

<br/>

## Load some data


Let's load some data: an indoor temperature winter dataset.

In [2]:
from timeseria import storages

# Set sataset path to Timeria test CSV data
DATASET_PATH = '/'.join(storages.__file__.split('/')[0:-1]) + '/tests/test_data/csv/'

# Create a CSV storage pointing to out dataset
csv_storage = storages.CSVFileStorage(DATASET_PATH + 'temperature_winter.csv')

# Get the time series out from the CSV file storage 
temperature_timeseries = csv_storage.get()

Let's have a look at the time series we just loaded:

In [3]:
temperature_timeseries

Time series of #14403 points at variable resolution (~10m), from point @ 1546477200.0 (2019-01-03 01:00:00+00:00) to point @ 1555550400.0 (2019-04-18 01:20:00+00:00)

Now plot the data, using Timeseria built-in plotting engine:

In [4]:
temperature_timeseries.plot()

[INFO] timeseria.plots: Aggregating by "10" for improved plotting


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In this plot, data has been aggregated by a factor of 10, and represented as a line chart plus an area chart (for the minimum and maximum values). In this way you can plot even millions of data points wihtout slowing down the plot engine or crashing your browser, without loosing much information about peaks and spikes.

You can also generate image-based, non-interactive plots by using `plot(image=True)`. In this case, if not already done, Timeseria will automatically download a Chromium headless engine, which is requiredfor rendering this kind of plots.

<br/>

## Resample (and make data uniform)

Now resample the time series at one hour sampling interval, also making data uniform and equally spaced over time. Gaps are filled by linear interpolation (the default interpolation method for the resampling) and the "data_loss" index is added to the data indexes and as a point attribute as well.

In [5]:
temperature_timeseries = temperature_timeseries.resample('1h')

[INFO] timeseria.transformations: Using auto-detected sampling interval: 600.0s
[INFO] timeseria.transformations: Resampled 14403 DataTimePoints in 2519 DataTimePoints


In [6]:
temperature_timeseries

Time series of #2519 points at 1h resolution, from point @ 1546480800.0 (2019-01-03 02:00:00+00:00) to point @ 1555545600.0 (2019-04-18 00:00:00+00:00)

In [7]:
temperature_timeseries[58].data_loss

1.0

In [8]:
temperature_timeseries.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br/>

## Reconstruct missing data

Train a periodic average-based data reconstruction model. Fit the model on about 80% of the data and test on the rest, on 6, 12, and 24 hours gaps, limiting the evaluation to 100 samples per step to speed up the process:

In [9]:
from timeseria.models import PeriodicAverageReconstructor

In [10]:
paverage_reconstructor = PeriodicAverageReconstructor()
paverage_reconstructor.fit(temperature_timeseries[0:2000])
paverage_reconstructor.evaluate(temperature_timeseries[2000:], steps=[6,12,24], limit=100)

[INFO] timeseria.models.reconstructors: Detected periodicity: 24x 1h
[INFO] timeseria.models.reconstructors: Will evaluate model for [6, 12, 24] steps with metrics ['RMSE', 'MAE']


{'RMSE': 0.7231735867454948, 'MAE': 0.5890984905648669}

Use the reconstruction model to reconstruct missing data points (100% data loss). A "data_reconstructed" index is added to the data indexes.

In [11]:
temperature_timeseries = paverage_reconstructor.apply(temperature_timeseries)

In [12]:
temperature_timeseries[58].data_indexes['data_reconstructed']

1

...and plot:

In [13]:
temperature_timeseries.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br />

## Three days hourly temperature forecast

Use a LSTM neural network model to to forecast three days of temperatures, but before fitting run a cross validation on three rounds to get an idea about the accuracy to expect, limiting the evaluation to 3 rounds and 100 samples per step to speed up the process:

In [14]:
from timeseria.models import LSTMForecaster

In [15]:
LSTM_forecaster = LSTMForecaster(window=12, neurons=64, features=['values', 'diffs', 'hours'])
LSTM_forecaster.cross_validate(temperature_timeseries[0:100], rounds=3)

[INFO] timeseria.models.base: Cross validation round #1 of 3: validate from 1546480800.0 (2019-01-03 02:00:00+00:00) to 1546599600.0 (2019-01-04 11:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Will evaluate model for [1, 12] steps ahead with metrics ['RMSE', 'MAE']
[INFO] timeseria.models.base: Cross validation round #2 of 3: validate from 1546599600.0 (2019-01-04 11:00:00+00:00) to 1546718400.0 (2019-01-05 20:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Will evaluate model for [1, 12] steps ahead with metrics ['RMSE', 'MAE']
[INFO] timeseria.models.base: Cross validation round #3 of 3: validate from 1546718400.0 (2019-01-05 20:00:00+00:00) to 1546837200.0 (2019-01-07 05:00:00+00:00), fit on the rest.
[INFO] timeseria.models.forecasters: Will evaluate model for [1, 12] steps ahead with metrics ['RMSE', 'MAE']


{'RMSE_avg': 0.3899099422038092,
 'RMSE_stdev': 0.1220358100988712,
 'MAE_avg': 0.3001876728576967,
 'MAE_stdev': 0.10587180782318344}

Now fit, apply for 72 steps ahead and plot, but only displaying the "forecast" data index (added by the forecaster):

In [16]:
LSTM_forecaster.fit(temperature_timeseries)
LSTM_forecaster.apply(temperature_timeseries,steps=72).plot(data_indexes=['forecast'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


<br />

## Run an anomaly detection model

Use the periodic average anomaly detection model, which will consider a data point value an anomaly if too far from its periodic average. This will add an "anomaly" index as data points attibute.

In [17]:
from timeseria.models import PeriodicAverageAnomalyDetector
paverage_anomaly_detector = PeriodicAverageAnomalyDetector()
paverage_anomaly_detector.fit(temperature_timeseries, stdevs=5)

[INFO] timeseria.models.forecasters: Detected periodicity: 24x 1h
[INFO] timeseria.models.forecasters: Using a window of "24"
[INFO] timeseria.models.anomaly_detectors: Using 5 standard deviations as anomaly threshold: 1.8318256468371585


Apply and plot, but show only the "anomaly" index to improve plot readability:

In [18]:
paverage_anomaly_detector.apply(temperature_timeseries).plot(data_indexes=['anomaly'])

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br/>

## Altogether

Apply again both models, but this time on the same time series, and plot:

In [19]:
anoamaly_temperature_timeseries = paverage_anomaly_detector.apply(temperature_timeseries)
forecast_and_anomaly_temperature_timeseries = LSTM_forecaster.apply(anoamaly_temperature_timeseries, steps=72)
forecast_and_anomaly_temperature_timeseries.plot()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br/>

## Move to daily data

Aggregate the time series into 1-day slots, also computing the min and max operations besideds the (default) average one, but change the timezone before, to get the right daily aggregates and to properly take into account the DST change. Slotting in days is indeed different than slotting in 24 hours, they are just two different time units: the first is variable, the second is fixed.

In [20]:
forecast_and_anomaly_temperature_timeseries.change_timezone('Europe/Rome')
from timeseria.operations import min, max, avg
forecast_and_anomaly_temperature_timeseries.aggregate(unit='1D', operations=[min,max,avg]).plot()

[INFO] timeseria.transformations: Using auto-detected sampling interval: 3600.0s
[INFO] timeseria.transformations: Aggregated 2566 points in 106 slots


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<br/>

## Next steps

You can have a look at the [welcome notebook](https://nbviewer.org/github/sarusso/Timeseria-notebooks/blob/master/notebooks/Welcome.ipynb), which provides a more structured introduction on Timeseria datastructures and their philosophy, as well as practical examples on both built-in and custom models and operations.


You can also try yourself by running the Timeseria Docker container, wich comes with this example notebooks repository pre-installed:

    docker run -it -p8888:8888 sarusso/timeseria


Or, you can give it a try in your own projects:

    pip install timeseria
    

..and you can of course contribute! :)
<br/>
<br/>


